# Lecture 1: Breaking RSA with SPA - Automate

In this notebook we want to see how to develop an algorithm which automatically reveals the exponent from a given trace.

In [ ]:
import securec
import securec.util as util
scope, target = util.init()

In [ ]:
securec.util.compile_and_flash('./2_rsa_uint8_variable.c')

In [ ]:
import struct
import time
import warnings
    
scope.default_setup()
scope.adc.samples = 5500

def capture(exponent, message=0xA0, modulus=0xFF):
    scope.arm()
    target.simpleserial_write('r', struct.pack('<BBB', message, exponent, modulus))
    return util.capture()

## Recap

First of all we want to recap the recorded traces from the last notebook.

In [ ]:
from bokeh.plotting import figure, show 
from bokeh.io import output_notebook
from bokeh.models import CrosshairTool
from bokeh.palettes import Category10_10

output_notebook()

In [ ]:
traces = [(exp, capture(exp)) for exp in (0b100, 0b101, 0b111)]

In [ ]:
p = figure(width=900, height=300)
p.add_tools(CrosshairTool())
for i, (exp, trace) in enumerate(traces):
    p.line(range(0, len(trace)), trace - i * 0.6, legend_label=bin(exp), line_color=Category10_10[i])
show(p)

## A bit of signal processing

In [ ]:
import numpy as np
import scipy
import scipy.signal

### Crop
First of all we want to crop the traces so that we really concentrate on the proper values.

In [ ]:
def crop(trace, peak_height=0.4):
    """Cut off irrelevant parts of given trace by looking at the most significant peaks."""
    peaks, _ = scipy.signal.find_peaks(-trace, height=peak_height)
    return trace[peaks[0]:peaks[1]]

In [ ]:
p = figure(width=900, height=300)
p.add_tools(CrosshairTool())
for i, (exp, trace) in enumerate(traces):
    trace = crop(trace)
    p.line(range(0, len(trace)), trace - i * 0.6, legend_label=bin(exp), line_color=Category10_10[i])
show(p)

### Preprocess

With manual analysis we can recognize that the peaks look a bit different if the processed bit is 0 or 1. But to make this "visible" for an algorithm let's consider the following modifications.   

In [ ]:
def neighbormax(trace, neighbors=50):
    """Calculate the pointwise maximum of a trace respecting each point's neighborhood"""
    trace = np.abs(trace)
    return np.array([np.max(trace[i:i + neighbors]) for i in range(len(trace))])

In [ ]:
p = figure(width=900, height=300)
p.add_tools(CrosshairTool())
for i, (exp, trace) in enumerate(traces):
    trace = neighbormax(crop(trace))
    p.line(range(0, len(trace)), trace - i * 0.3, legend_label=bin(exp), line_color=Category10_10[i])
show(p)

### Detecting peaks

Now it's quite obvious where 0 and 1 occur. Each of the down-peaks mark the start of a new multiplication round.

In [ ]:
def downpeaks(trace, peak_height=0.2, digest_width=800):
    """Returning peaks below given height. 
    Note: First an last point of trace are also considered as peaks."""
    peaks, _ = scipy.signal.find_peaks(-trace, height=-peak_height)
    peaks = [p for p in peaks if p > digest_width / 2 and p < len(trace) - digest_width / 2]
    peaks = [0] + peaks + [len(trace) - 1]
    return np.array(peaks), trace[peaks]

In [ ]:
p = figure(width=900, height=300)
p.add_tools(CrosshairTool())
for i, (exp, trace) in enumerate(traces):
    trace = neighbormax(crop(trace))
    p.line(range(0, len(trace)), trace - i * 0.3, legend_label=bin(exp), line_color=Category10_10[i])
    trace_peaks = downpeaks(trace)
    p.circle(trace_peaks[0], trace_peaks[1] - i * 0.3, color=Category10_10[i])
show(p)

## Writing the algorithm

Now it's quite clear how to proceed: Look at the distance between two peaks. If it is equal to 1 multiplication the exponent's bit was 0, if it is 2 the exponent's bit was 1.

In [ ]:
def attack_rsa_spa(trace, digest_width=800, max_neighbors=50, downpeak_height=0.2):
    """
    Reveal exponent's bit by analyzing given trace.

    :param digest_width: With of one multiplication
    :param max_neighbors: Number of neighbors to look at when computing pointwise max
    :param downpeak_height: Height of peak for detecting peaks
    
    :return: List of 0 and 1 indicating bits of exponent
    """
    # Preprocess trace
    trace = neighbormax(crop(trace), neighbors=max_neighbors)
    # Find downpeaks including start and end point
    peak, _ = downpeaks(trace, peak_height=downpeak_height, digest_width=digest_width)
    # Use information of digest width to convert to binary
    normalized_peaks = [int(p) for p in np.round(peak / digest_width)]
    return list(reversed([0 if p1 + 1 == p2 else 1 for p1, p2 in zip(normalized_peaks, normalized_peaks[1:])]))
    

In [ ]:
for exp, trace in traces:
    print(bin(exp), '=>', attack_rsa_spa(trace))

In [ ]:
util.exit()